In [1]:
import datetime
import pandas as pd

In [2]:
def na_table(x):
    na = pd.DataFrame(x.isna().sum())
    nna = pd.DataFrame(x.notna().sum())
    na.reset_index(inplace=True)
    nna.reset_index(inplace=True)
    na = na[na[0]>0]
    na = na.merge(nna, on='index')
    na.columns = ['col', 'isNA', 'notNA']
    return na

In [3]:
data_dir = '../train/'
features_dir = '../features/'
news_info = pd.read_csv(data_dir + 'news_info.csv',encoding = 'gbk')
na = na_table(news_info)
na

,col,isNA,notNA


In [4]:
base_info = pd.read_csv(data_dir + 'base_info.csv',encoding='gbk')
base_info.drop_duplicates(inplace=True)
base_info.opfrom = pd.to_datetime(base_info.opfrom)
base_info.opfrom = base_info.opfrom.apply(lambda x:datetime.datetime.now() - x)
base_info.opfrom = base_info.opfrom.apply(lambda x:x.days/365)
base_info = pd.DataFrame([base_info.id, base_info.opfrom]).T
base_info.columns = ['id', 'years']
base_info.head()

,id,years
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,1.29589
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,3.13973
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,0.115068
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,5.07671
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2.90411


In [5]:
news_info['year'] = news_info.public_date.apply(lambda x:x.split('-')[0])
news_info = pd.DataFrame(news_info.groupby(['id']).apply(lambda x:x.positive_negtive.value_counts()))
news_info.reset_index(inplace=True)
news_info.columns = ['id', 'positive_negtive', 'counts']
news_info = news_info.merge(base_info, on='id')
news_info.years =news_info.years.astype('int')
news_info['avg_news'] = news_info.counts / news_info.years
news_info.drop(columns=['years'], inplace=True)
news_info.head()

,id,positive_negtive,counts,avg_news
0,09912c34159b1720558a419983a989f1dd2e0ed69a044ca3,中立,6,1.200000
1,175ebe5f059ec050afbd65251ecdd3b512bfbe5e62d041b0,积极,4,0.444444
2,175ebe5f059ec050afbd65251ecdd3b512bfbe5e62d041b0,中立,3,0.333333
3,216bd2aaf4d079240c3ac0b76f0ef4aa355d443880ba78db,积极,2,2.000000
4,216bd2aaf4d079240c3ac0b76f0ef4aa355d443880ba78db,中立,1,1.000000


In [6]:
news_info.pivot_table(index='id', columns='positive_negtive')

avg_news            \
positive_negtive                                        中立        消极   
id                                                                     
09912c34159b1720558a419983a989f1dd2e0ed69a044ca3  1.200000       NaN   
175ebe5f059ec050afbd65251ecdd3b512bfbe5e62d041b0  0.333333       NaN   
216bd2aaf4d079240c3ac0b76f0ef4aa355d443880ba78db  1.000000       NaN   
216bd2aaf4d079240f5823e63d24b44dd2c58e3281b822f6  2.000000       NaN   
216bd2aaf4d0792410725ba5e7ca1dc32ce55767372f2030       NaN  0.062500   
...                                                    ...       ...   
f000950527a6feb6fddefb42e3c3dc1932fc8c5fae14afbb  0.125000       NaN   
f000950527a6feb6fe8f4850e9eb04e8ba3fa3b409725ef3  0.222222       NaN   
f000950527a6feb6ff749dc50c7bf46b37b74e36ce38d1a4       NaN  0.076923   
f000950527a6feb6ff7cdb55f5e64a477c499dd75137ae6b       NaN       NaN   
f000950527a6feb6ff839cdf509ebb7631857e6e363fedd6  0.055556  0.055556   

                                                           counts            
positive_negtive                                        积极     中立   消极   积极  
id                                                                           
09912c34159b1720558a419983a989f1dd2e0ed69a044ca3       NaN    6.0  NaN  NaN  
175ebe5f059ec050afbd65251ecdd3b512bfbe5e62d041b0  0.444444    3.0  NaN  4.0  
216bd2aaf4d079240c3ac0b76f0ef4aa355d443880ba78db  2.000000    1.0  NaN  2.0  
216bd2aaf4d079240f5823e63d24b44dd2c58e3281b822f6       NaN    2.0  NaN  NaN  
216bd2aaf4d0792410725ba5e7ca1dc32ce55767372f2030       NaN    NaN  1.0  NaN  
...                                                    ...    ...  ...  ...  
f000950527a6feb6fddefb42e3c3dc1932fc8c5fae14afbb       NaN    1.0  NaN  NaN  
f000950527a6feb6fe8f4850e9eb04e8ba3fa3b409725ef3  0.777778    2.0  NaN  7.0  
f000950527a6feb6ff749dc50c7bf46b37b74e36ce38d1a4       NaN    NaN  1.0  NaN  
f000950527a6feb6ff7cdb55f5e64a477c499dd75137ae6b  0.142857    NaN  NaN  2.0  
f000950527a6feb6ff839cdf509ebb7631857e6e363fedd6       NaN    1.0  1.0  NaN  

[927 rows x 6 columns]

In [7]:
news_info = news_info.pivot_table(index='id', columns='positive_negtive').fillna(0)
news_info.head()

avg_news                    \
positive_negtive                                        中立      消极        积极   
id                                                                             
09912c34159b1720558a419983a989f1dd2e0ed69a044ca3  1.200000  0.0000  0.000000   
175ebe5f059ec050afbd65251ecdd3b512bfbe5e62d041b0  0.333333  0.0000  0.444444   
216bd2aaf4d079240c3ac0b76f0ef4aa355d443880ba78db  1.000000  0.0000  2.000000   
216bd2aaf4d079240f5823e63d24b44dd2c58e3281b822f6  2.000000  0.0000  0.000000   
216bd2aaf4d0792410725ba5e7ca1dc32ce55767372f2030  0.000000  0.0625  0.000000   

                                                 counts            
positive_negtive                                     中立   消极   积极  
id                                                                 
09912c34159b1720558a419983a989f1dd2e0ed69a044ca3    6.0  0.0  0.0  
175ebe5f059ec050afbd65251ecdd3b512bfbe5e62d041b0    3.0  0.0  4.0  
216bd2aaf4d079240c3ac0b76f0ef4aa355d443880ba78db    1.0  0.0  2.0  
216bd2aaf4d079240f5823e63d24b44dd2c58e3281b822f6    2.0  0.0  0.0  
216bd2aaf4d0792410725ba5e7ca1dc32ce55767372f2030    0.0  1.0  0.0

In [8]:
news_info.reset_index(inplace=True)
news_info.columns = ['id','avg_neu_news','avg_neg_news','avg_pos_news', 'neu_news','neg_news','pos_news']
news_info.head()

,id,avg_neu_news,avg_neg_news,avg_pos_news,neu_news,neg_news,pos_news
0,09912c34159b1720558a419983a989f1dd2e0ed69a044ca3,1.200000,0.0000,0.000000,6.0,0.0,0.0
1,175ebe5f059ec050afbd65251ecdd3b512bfbe5e62d041b0,0.333333,0.0000,0.444444,3.0,0.0,4.0
2,216bd2aaf4d079240c3ac0b76f0ef4aa355d443880ba78db,1.000000,0.0000,2.000000,1.0,0.0,2.0
3,216bd2aaf4d079240f5823e63d24b44dd2c58e3281b822f6,2.000000,0.0000,0.000000,2.0,0.0,0.0
4,216bd2aaf4d0792410725ba5e7ca1dc32ce55767372f2030,0.000000,0.0625,0.000000,0.0,1.0,0.0


In [9]:
news_info.set_index(keys='id', inplace=True)
news_info.to_csv(features_dir + 'news_info.csv')